In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pylab as pl
import scipy as sp
import sys

import os, sys

%matplotlib inline

In [2]:
from box import Box
import glob

In [10]:
TOP = os.getcwd().replace("notebooks", "")
D1 = TOP + 'data/raw/'
figures_dir = TOP + 'reports/figures/'


In [12]:
CHM1 = pd.read_csv(D1 + 'S7-chem.tsv', sep='\t', low_memory=False)
CHM1 = CHM1.replace(['-'], np.nan)
CHM1.isnull().values.any()
CHM1 = CHM1.dropna()
CHM1
CHM1_nm = CHM1[['DTXSID', 'PREFERRED_NAME']].set_index('DTXSID')
X = CHM1 = CHM1.drop(['PREFERRED_NAME'], axis=1).set_index('DTXSID')

CHM11 = Box()
for i in set([i[0] for i in X.columns.str.split('_') ]):
    CHM11[i] =  X[X.columns[X.columns.str.match(i)]]
CHM11['all'] = X

In [13]:
BIO1 = pd.read_csv(D1+'S2-bio.tsv', sep='\t') 
BIO1 = BIO1.rename(columns = {'dsstox_substance_id':'DTXSID'})

BIO1 = BIO1.drop(['chnm', 'casn'], axis = 1).set_index('DTXSID')
BIO11 = Box()
for i in set([i[0] for i in BIO1.columns.str.split('_')]):
    BIO11[i]=BIO1[BIO1.columns[BIO1.columns.str.match(i)]]
BIO11['all']=BIO1

In [14]:
TOX1 = pd.read_csv(D1+'S1-tox.tsv', sep='\t')

TOX1 = TOX1.rename(columns = {'dsstox_substance_id':'DTXSID'})
TOX1 = TOX1.drop(['preferred_name'], axis=1).set_index('DTXSID')
#set([i[0] for i in TOX1.columns.str.split('_')])
TOX11 = Box()
for i in set([i[0] for i in TOX1.columns.str.split('_')]):
    TOX11[i] = TOX1[TOX1.columns[TOX1.columns.str.match(i)]]
TOX11['all'] = TOX1

In [15]:
DATA = Box(chm=CHM11,bio=BIO11,tox=TOX11)
DATA['bc'] = Box()

for c,b in [(i,j) for i in CHM11.keys() for j in BIO11.keys()]:
    print(b,c)
    DATA['bc']['{}_{}'.format(b,c)] = CHM11[c].merge(BIO11[b],left_index=True,right_index=True)

assay tptr
gene tptr
all tptr
assay mrgn
gene mrgn
all mrgn
assay toxp
gene toxp
all toxp
assay all
gene all
all all


In [16]:
effects = ['chr_liver', 'dev_liver']

In [17]:
def getData(b,dt='chm',st='all',Data=DATA):
    """
    Return an input and output dataframe for analysis
    
    b = toxicity class
    dt= input data type (chm or bio)
    st= inpute subdata type ()
    """
    
    Y = Data.tox.all[b].dropna()
    INT=Data[dt][st].index.intersection(Y.index)
    if len(INT)>0:
        Yf= Y.copy()[INT]
        Yf[Yf>0]=1
        Yb = Yf.copy()
        Yb = Yb.astype(np.int32)
        X  = Data[dt][st].loc[INT].fillna(0)
    else:
        pass
    
    return X,Yb

In [18]:
mydict = {}
for e in effects:
    data = getData(e,dt='chm',st='mrgn')
    mydict[e] = data


In [19]:
from sklearn.feature_selection import SelectKBest,f_classif,chi2
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import BernoulliNB
#from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import f_classif, f_regression
from sklearn.feature_selection import SelectKBest, SelectPercentile
from genra.rax.skl.cls import *
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.metrics import make_scorer, recall_score, f1_score, accuracy_score, precision_score
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif


## CHI  FS

In [20]:
#mydict

## ML classification

In [21]:
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import BernoulliNB
#from sklearn.tree import DecisionTreeClassifier

from genra.rax.skl.cls import *



In [22]:
for i in range(10,90,10):
    print(i)

10
20
30
40
50
60
70
80


In [23]:
def model_cv(d,  label = None):
    Res = []
    Est = [('Random Forest', RandomForestClassifier(random_state=42)),
          ('NB', BernoulliNB()),
          ('LR', LogisticRegression(max_iter=1000)),
          ('Gradient Boosting', GradientBoostingClassifier()),
          ('SVC', SVC(gamma='auto',probability=True)),
          ('KNN',KNeighborsClassifier()),
          ('ANN1',MLPClassifier(solver='sgd')),
          ('GenRA', GenRAPredClass(n_neighbors=10,metric='jaccard'))]
   
    X, Y = d
    for n in range(10,90,10):
        for model in Est:
            LR, Clf = model
            steps = [('variance_thresh', VarianceThreshold(threshold=0.01)), ('feature_selection', SelectKBest(score_func=chi2, k = n)), model]
            pipeline = Pipeline(steps=steps)
            score = cross_validate(pipeline, X, Y,
                               cv=StratifiedKFold(n_splits = 5),
                               scoring= {'accuracy': make_scorer(accuracy_score),
                                         'f1': make_scorer(f1_score),
                                         'sensitivity': make_scorer(recall_score),
                                         'specificity': make_scorer(recall_score,pos_label=0),
                                        'precision': make_scorer(precision_score)},
                               n_jobs=-1, verbose=1)
        #elapsed_time = datetime.now() - start_time
            SC = pd.DataFrame(score)
            SC.insert(0,'LR',LR)
            SC.insert(1, 'k', n)
            Res.append(SC)
    return Res

In [24]:
def plugin(d, label=None):
    summary = {}
    X, Y = d
    summary['label'] = label
    summary['score'] = model_cv(d)
    
    return summary


In [25]:
chm_summaries = []
for k in list(mydict.keys()):
    try:
        chm_summaries.append(plugin(mydict[k], k))
    except Exception as error:
        print(k)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    1.6s remaining:    2.4s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_

In [27]:
#chm_summaries[0]

In [28]:
myperm2 = {}
myperm = {}
for i in range(len(chm_summaries)):
    k = chm_summaries[i]['label']
    Perf = pd.concat(chm_summaries[i]['score'])
    Perf_agg = Perf.groupby(['LR']).aggregate(dict(test_f1=[np.mean,np.std],
                                    test_sensitivity=[np.mean,np.std],
                                    test_specificity=[np.mean,np.std],
                                    test_accuracy=[np.mean,np.std],
                                    test_precision=[np.mean,np.std],))\
    .round(decimals=3)
    myperm[k] = Perf_agg
    myperm2[k] = Perf       


In [29]:
Perf_mgrn = pd.concat(myperm)    
Perf_mgrn_raw = pd.concat(myperm2)

In [29]:
#Perf_mgrn_raw

Model Validation Using Bio (gene hitc) Descriptors

In [30]:
bio_dict = {}
for e in effects:
    data = getData(e,dt='bio',st='gene')
    bio_dict[e] = data


In [31]:
bio_summaries = []
for k in list(bio_dict.keys()):
    try:
        bio_summaries.append(plugin(bio_dict[k], k))
    except Exception as error:
        print(k)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_

In [32]:
bio_perm2 = {}
bio_perm = {}
for i in range(len(bio_summaries)):
    k = bio_summaries[i]['label']
    Perf = pd.concat(bio_summaries[i]['score'])
    Perf_agg = Perf.groupby(['LR']).aggregate(dict(test_f1=[np.mean,np.std],
                                    test_sensitivity=[np.mean,np.std],
                                    test_specificity=[np.mean,np.std],
                                    test_accuracy=[np.mean,np.std],
                                    test_precision=[np.mean,np.std],))\
    .round(decimals=3)
    bio_perm[k] = Perf_agg
    bio_perm2[k] = Perf


In [33]:
Perf_bio = pd.concat(bio_perm)  
Perf_bio_raw = pd.concat(bio_perm2)  

Model Validation Using Hybrid (chm(mrgn) + bio(hitc)) Descriptors

In [34]:
hybrid_dict = {}
for e in effects:
    data = getData(e,dt='bc',st='gene_mrgn')
    hybrid_dict[e] = data

In [35]:
hybrid_summaries = []
for k in list(hybrid_dict.keys()):
    try:
        hybrid_summaries.append(plugin(hybrid_dict[k], k))
    except Exception as error:
        print(k)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.2s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_

In [36]:
myperm2 = {}
myperm = {}
for i in range(len(hybrid_summaries)):
    k = hybrid_summaries[i]['label']
    Perf = pd.concat(hybrid_summaries[i]['score'])
    Perf_agg = Perf.groupby(['LR']).aggregate(dict(test_f1=[np.mean,np.std],
                                    test_sensitivity=[np.mean,np.std],
                                    test_specificity=[np.mean,np.std],
                                    test_accuracy=[np.mean,np.std],
                                    test_precision=[np.mean,np.std],))\
    .round(decimals=3)
    myperm[k] = Perf_agg
    myperm2[k] = Perf

In [37]:
Perf_hybrid = pd.concat(myperm)  
Perf_hybrid_raw = pd.concat(myperm2) 


Model Validation Using All Chm (mrgn + tptr + toxp) Descriptors

In [38]:
ca_dict = {}
for e in effects:
    data = getData(e,dt='chm',st='all')
    ca_dict[e] = data


In [39]:
ca_summaries = []
for k in list(ca_dict.keys()):
    try:
        ca_summaries.append(plugin(ca_dict[k], k))
    except Exception as error:
        print(k)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.6s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.4s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.5s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.5s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_

In [40]:
ca_perm2 = {}
ca_perm = {}
for i in range(len(ca_summaries)):
    k = ca_summaries[i]['label']
    Perf = pd.concat(ca_summaries[i]['score'])
    Perf_agg = Perf.groupby(['LR']).aggregate(dict(test_f1=[np.mean,np.std],
                                    test_sensitivity=[np.mean,np.std],
                                    test_specificity=[np.mean,np.std],
                                    test_accuracy=[np.mean,np.std],
                                    test_precision=[np.mean,np.std],))\
    .round(decimals=3)
    ca_perm[k] = Perf_agg
    ca_perm2[k] = Perf

In [41]:
Perf_ca = pd.concat(ca_perm)  
Perf_ca_raw = pd.concat(ca_perm2)  

Model Validation Using All Chm + bio (mrgn + tptr + toxp + bio) Descriptors

In [42]:
bc_dict = {}
for e in effects:
    data = getData(e,dt='bc',st='gene_all')
    bc_dict[e] = data

In [43]:
bc_summaries = []
for k in list(bc_dict.keys()):
    try:
        bc_summaries.append(plugin(bc_dict[k], k))
    except Exception as error:
        print(k)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    1.7s remaining:    2.6s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.7s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.5s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    0.6s remaining:    0.9s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_

In [44]:
bc_perm2 = {}
bc_perm = {}
for i in range(len(bc_summaries)):
    k = bc_summaries[i]['label']
    Perf = pd.concat(bc_summaries[i]['score'])
    Perf_agg = Perf.groupby(['LR']).aggregate(dict(test_f1=[np.mean,np.std],
                                    test_sensitivity=[np.mean,np.std],
                                    test_specificity=[np.mean,np.std],
                                    test_accuracy=[np.mean,np.std],
                                    test_precision=[np.mean,np.std],))\
    .round(decimals=3)
    bc_perm[k] = Perf_agg
    bc_perm2[k] = Perf

In [45]:
Perf_bc = pd.concat(bc_perm)  
Perf_bc_raw = pd.concat(bc_perm2)  

In [46]:
def pref_files(df, dt, st, bl):
    df = df.reset_index()
    df.columns = ['tox', 'classifier', 'mean_f1', 'std_f1', 'mean_sensitivity', 'std_sensitivity', 'mean_specificity', 'std_specificity', 'mean_accuracy', 'std_accuracy', 'mean_precision', 'std_precision']
    #df = df.drop([0,1])
    df['dt'] = dt 
    df['balance'] = bl
    df['sampling'] = st
    #df = pd.merge(df, df_counts, on = 'tox', how = 'left')
    return df

In [47]:
mylst = ['mrgn', 'gene', 'cb', 'ca', 'cba']

In [48]:
mylst2 = list(zip(mylst, [Perf_mgrn, 
Perf_bio, 
Perf_hybrid,
Perf_ca, 
Perf_bc ], ))


In [49]:
mylst3 = []
for e in mylst2:
    a,b = e
    mylst3.append(pref_files(b,a, st = 'unbalanced', bl = 'unbalanced'))

In [50]:
final_df = pd.concat(mylst3)

In [51]:
final_df

,tox,classifier,mean_f1,std_f1,mean_sensitivity,std_sensitivity,mean_specificity,std_specificity,mean_accuracy,std_accuracy,mean_precision,std_precision,dt,balance,sampling
0,chr_liver,ANN1,0.786,0.006,0.998,0.011,0.001,0.006,0.647,0.009,0.648,0.006,mrgn,unbalanced,unbalanced
1,chr_liver,GenRA,0.723,0.061,0.783,0.151,0.329,0.235,0.623,0.040,0.693,0.052,mrgn,unbalanced,unbalanced
2,chr_liver,Gradient Boosting,0.741,0.033,0.856,0.089,0.171,0.107,0.615,0.033,0.656,0.020,mrgn,unbalanced,unbalanced
3,chr_liver,KNN,0.720,0.061,0.785,0.155,0.307,0.245,0.617,0.042,0.686,0.053,mrgn,unbalanced,unbalanced
4,chr_liver,LR,0.748,0.023,0.867,0.072,0.173,0.111,0.623,0.023,0.660,0.019,mrgn,unbalanced,unbalanced
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,dev_liver,KNN,0.024,0.072,0.017,0.047,0.983,0.026,0.825,0.022,0.054,0.183,cba,unbalanced,unbalanced
12,dev_liver,LR,0.056,0.123,0.039,0.086,0.974,0.017,0.821,0.017,0.100,0.223,cba,unbalanced,unbalanced
13,dev_liver,NB,0.146,0.128,0.116,0.102,0.928,0.036,0.795,0.034,0.214,0.203,cba,unbalanced,unbalanced
14,dev_liver,Random Forest,0.087,0.099,0.076,0.093,0.925,0.051,0.786,0.037,0.112,0.129,cba,unbalanced,unbalanced


In [52]:
def pref_files2(df, dt, st, bl):
    df = df.reset_index()
    df = df.drop(['level_1'], axis = 1)
    df.columns = ['tox', 'classifier', 'kfeats', 'fit_time', 'score_time',
       'test_accuracy', 'test_f1', 'test_sensitivity', 'test_specificity',
       'test_precision']
    #df = df.drop([0,1])
    df['dt'] = dt 
    df['balance'] = bl
    df['sampling'] = st
    #df = pd.merge(df, df_counts, on = 'tox', how = 'left')
    return df

In [53]:
mylst4 = list(zip(mylst, [Perf_mgrn_raw, 
Perf_bio_raw, 
Perf_hybrid_raw,
Perf_ca_raw, 
Perf_bc_raw ], ))


In [54]:
mylst5 = []
for e in mylst4:
    a,b = e
    mylst5.append(pref_files2(b,a, st = 'unbalanced', bl = 'unbalanced'))

In [55]:
all_df = pd.concat(mylst5)

In [56]:
all_df

,tox,classifier,kfeats,fit_time,score_time,test_accuracy,test_f1,test_sensitivity,test_specificity,test_precision,dt,balance,sampling
0,chr_liver,Random Forest,10,0.208705,0.031783,0.643836,0.775862,0.937500,0.080000,0.661765,mrgn,unbalanced,unbalanced
1,chr_liver,Random Forest,10,0.202618,0.028257,0.671233,0.796610,1.000000,0.076923,0.661972,mrgn,unbalanced,unbalanced
2,chr_liver,Random Forest,10,0.166047,0.019354,0.616438,0.762712,0.957447,0.000000,0.633803,mrgn,unbalanced,unbalanced
3,chr_liver,Random Forest,10,0.188608,0.018500,0.657534,0.786325,0.978723,0.076923,0.657143,mrgn,unbalanced,unbalanced
4,chr_liver,Random Forest,10,0.191557,0.028305,0.638889,0.775862,0.957447,0.040000,0.652174,mrgn,unbalanced,unbalanced
...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,dev_liver,GenRA,80,0.243845,0.047482,0.811321,0.000000,0.000000,0.977273,0.000000,cba,unbalanced,unbalanced
636,dev_liver,GenRA,80,0.268283,0.044435,0.811321,0.000000,0.000000,0.977273,0.000000,cba,unbalanced,unbalanced
637,dev_liver,GenRA,80,0.275121,0.046910,0.846154,0.000000,0.000000,1.000000,0.000000,cba,unbalanced,unbalanced
638,dev_liver,GenRA,80,0.261438,0.026728,0.846154,0.000000,0.000000,1.000000,0.000000,cba,unbalanced,unbalanced


In [57]:
writer = pd.ExcelWriter(D1+'liver_unbalanced_all_featuresel_140124.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.

final_df.to_excel(writer, sheet_name = 'all')
all_df.to_excel(writer, sheet_name = 'raw')
writer.save()